In [2]:
"""
UHID Essex County - Land Surface Temperature Analysis
Author: Samuel
Date: November 2025
Purpose: Process existing Landsat thermal imagery to extract LST hotspots

This notebook:
1. Loads pre-downloaded Landsat data from Google Drive
2. Processes thermal bands to Land Surface Temperature
3. Creates hotspot maps
4. Exports processed temperature grids
"""
print("Notebook: 01_LST_Analysis.ipynb loaded.")


Notebook: 01_LST_Analysis.ipynb loaded.


In [3]:
# ============================================
# SETUP - Mount Drive and Navigate
# ============================================
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/SharedFilesUHIDTeam')

print("✓ Drive mounted and navigated to data folder")

Mounted at /content/drive
✓ Drive mounted and navigated to data folder


In [4]:
# ============================================
# INSTALL & IMPORT LIBRARIES
# ============================================
print("\n=== Installing Required Packages ===")
!pip install rasterio geopandas matplotlib numpy scipy -q

import rasterio
from rasterio.plot import show
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✓ All packages loaded")



=== Installing Required Packages ===
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 94.0 MB/s eta 0:00:00
✓ All packages loaded


In [5]:
# ============================================
# STEP 1: EXPLORE LANDSAT DATA FOLDER
# ============================================
print("\n=== Checking Landsat Data Folder ===")

lst_folder = Path('01_Satellite_LST')

# List all files in the folder
if lst_folder.exists():
    files = list(lst_folder.glob('*'))
    print(f"✓ Found {len(files)} files in 01_Satellite_LST/")
    print("\nFiles:")
    for f in files[:20]:  # Show first 20 files
        print(f"  - {f.name} ({f.stat().st_size / 1e6:.1f} MB)")

    if len(files) > 20:
        print(f"  ... and {len(files) - 20} more files")
else:
    print("✗ 01_Satellite_LST folder not found!")


=== Checking Landsat Data Folder ===
✓ Found 1 files in 01_Satellite_LST/

Files:
  - Essex_County_LST_Summer_2023.tif (3.7 MB)


In [6]:
# ============================================
# STEP 2: IDENTIFY THERMAL BAND FILES
# ============================================
print("\n=== Identifying Thermal Band Files ===")

# Landsat thermal bands are typically:
# - Landsat 8/9: Band 10 (B10) - 100m resolution
# - File naming: LC08_...B10.TIF or LC09_...B10.TIF

thermal_files = []
for pattern in ['*B10*.TIF', '*B10*.tif', '*thermal*.tif', '*LST*.tif']:
    thermal_files.extend(list(lst_folder.glob(pattern)))

if thermal_files:
    print(f"✓ Found {len(thermal_files)} thermal band file(s)")
    for tf in thermal_files:
        print(f"  - {tf.name}")
else:
    print(" No thermal bands found with standard naming")
    print("\nPlease check file naming pattern. Looking for files containing:")
    print("  - 'B10' (Landsat Band 10)")
    print("  - 'thermal'")
    print("  - 'LST'")



=== Identifying Thermal Band Files ===
✓ Found 1 thermal band file(s)
  - Essex_County_LST_Summer_2023.tif
